# Calculate Node Weights
Calculate the relative weight (popularity) of every node in a boundary polygon in an effort to find possible starting points for routing.

In [26]:
import geopandas as gpd
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [27]:
# database connections
src_conn = psycopg2.connect(database='stc', user='postgres', password='postgres', host='localhost')
dest_conn = create_engine('sqlite:///seq.db')

In [28]:
# layer info
parent_uid = 'lu_uid'
parent_layer = 'lu'

In [32]:
sql = "SELECT {}, geom FROM {}".format(parent_uid, parent_layer)
pgeo = gpd.GeoDataFrame.from_postgis(sql, src_conn)

# pull out the nodes in the polygons
pgeo['coords'] = pgeo.geometry.boundary.apply(lambda x: x[0].coords)
sp = pgeo[[parent_uid, 'coords']]
d = []
for r in sp.iterrows():
    k = r[1][0]
    v = r[1][1]
    for i in v:
        d.append((k,i))
coord_pop = pd.DataFrame(d, columns=[parent_uid, 'node'])

coord_pop['weight'] = coord_pop.groupby(['node'])[parent_uid].transform('count')
coord_pop.drop_duplicates('node', inplace=True)

In [36]:
coord_pop.sort_values('weight').tail()

,lu_uid,node,weight
528915,68768,"(4037253.822900001, 1995076.8000000007)",3
453775,46819,"(5821987.614299998, 1541556.8456999995)",4
150894,15960,"(7627326.554300003, 1242333.6086000018)",4
285169,21911,"(7121664.748599999, 2355353.8143000007)",4
404099,34244,"(7203814.380000003, 905525.8200000003)",4


In [37]:
coord_pop['weight'].value_counts()

1    563545
2     22994
3       195
4         4
Name: weight, dtype: int64

In [12]:
coord_pop.dtypes

lu_uid     int64
node      object
weight     int64
dtype: object

In [38]:
coord_pop['node'] = coord_pop['node'].astype(str)

In [39]:
# write it all to sqlite for reference by later steps
coord_pop.to_sql('node_weights', con=dest_conn, if_exists='replace', index=False)